In [ ]:
import wandb

api = wandb.Api()
entity, project = "sayakpaul", "keras-xla-benchmarks"  
runs = api.runs(entity + "/" + project) 
print(f"Total runs: {len(runs)}")

In [ ]:
import pandas as pd

resolutions = []
accelerators = []

model_families = []
model_variants = []
xla_status = []

flops = []
params = []
throughputs = []

for run in runs:
    run_config = run.config
    run_summary = run.summary._json_dict

    model_families.append(run_config["family"])
    model_variants.append(run_config["variant"])
    resolutions.append(run_config["resolution"])
    xla_status.append(run_config["xla"])

    accelerator_name = run.name.split("@")[-1].split("-")[1]
    accelerators.append(accelerator_name)

    flops.append(run_summary["FLOPs (giga)"])
    params.append(run_summary["Num parameters (million)"])
    throughputs.append(run_summary["Throughput (samples/sec)"])

viz_df = pd.DataFrame(
    {
        "model_family": model_families,
        "model_variant": model_variants,
        "resolution": resolutions,
        "xla": xla_status,
        "accelerator": accelerators,
        "flop (giga)": flops,
        "params (million)": params,
        "throughput (samples/sec)": throughputs,
    }
)
viz_df.head()

In [ ]:
viz_df.to_csv("keras_xla_benchmarks.csv", index=False)

In [ ]:
# For the CNN model families, which one is the best?
# What about model families?
grouped = viz_df.groupby(["resolution", "model_family", "accelerator"])[
    "throughput (samples/sec)"
].idxmax()
result = viz_df.loc[grouped, viz_df.columns]
result

In [ ]:
result[result["accelerator"] == "a100"].sort_values(
    by=["throughput (samples/sec)"], ascending=False
)

In [ ]:
result.query("resolution == 224 and accelerator == 'a100'").sort_values(
    by=["throughput (samples/sec)"], ascending=False
)

In [ ]:
# Adapted from
# https://github.com/nlp-with-transformers/notebooks/blob/main/08_model-compression.ipynb

import matplotlib.pyplot as plt


def plot_metrics(df, savefig=False):
    for model_variant in df["model_variant"]:
        filtered = df.query(f"model_variant == '{model_variant}'")
        plt.scatter(
            filtered["flop (giga)"],
            filtered["throughput (samples/sec)"],
            alpha=0.5,
            s=filtered["params (million)"] * 5,
            label=model_variant,
            marker="o",
        )

    legend = plt.legend(bbox_to_anchor=(1, 1))
    for handle in legend.legendHandles:
        handle.set_sizes([20])

    plt.ylabel("Throughput (samples/sec)", fontsize=14)
    plt.xlabel("FLOPS (giga)", fontsize=14)

    accelerator_name = df["accelerator"].unique()[0]
    resolution = df["resolution"].unique()[0]
    xla_status = df["xla"].unique()[0]
    plt.title(
        f"Accelerator: {accelerator_name}, Resolution: {resolution}, XLA: {xla_status}",
        fontsize=14,
    )
    if not savefig:
        plt.show()
    else:
        plot_name = f"{accelerator_name}_{resolution}_{xla_status}.png"
        plt.savefig(plot_name, dpi=300, bbox_inches="tight")

In [ ]:
df_224_a100 = result.query("resolution == 224 and accelerator == 'a100'")

plot_metrics(df_224_a100)

In [ ]:
df_224_v100 = result.query("resolution == 224 and accelerator == 'v100'")

plot_metrics(df_224_v100)

In [ ]:
df_224_t4 = result.query("resolution == 224 and accelerator == 't4'")

plot_metrics(df_224_t4)